<a href="https://colab.research.google.com/github/vhrique/ELF/blob/main/20250131_Analise_Sucupira.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
planilhas = {
    '2021': 'https://drive.google.com/uc?export=download&id=1aY-35G2D8oFIwY9UtxaUP9K00Cz72Tf9',
    '2022': 'https://drive.google.com/uc?export=download&id=14QUdOCIMWLOL3De60GKPkvlgxzaMCDPz',
    '2023': 'https://drive.google.com/uc?export=download&id=1C2qkNL1N9B6Q6wUh33kbkQ8QRySnEfnZ',
    # '2024': 'https://drive.google.com/uc?export=download&id=1WLvc4wt8RsiN2Ob6UWjGq9SodFvQORCj',
}

In [3]:
df = {}
for ano in planilhas:
    df[ano] = pd.read_excel(planilhas[ano], sheet_name=None)

In [4]:
dfs = []
for ano in df:
    new_df = df[ano]['Projetos - Membros']
    new_df['Ano da Coleta'] = ano
    dfs.append(new_df)
df_projeto_membros = pd.concat(dfs).reset_index(drop=True)

In [5]:
dfs = []
for ano in df:
    new_df = df[ano]['Projetos - Financiadores']
    new_df['Ano da Coleta'] = ano
    dfs.append(new_df)
df_projeto_financiadores = pd.concat(dfs).reset_index(drop=True)

In [10]:
for i, row in df_projeto_financiadores.iterrows():
    df_projeto_financiadores.loc[i,'Nome do Projeto de Pesquisa'] = df_projeto_financiadores.loc[i,'Nome do Projeto de Pesquisa'].replace('"','')

for i, row in df_projeto_membros.iterrows():
    df_projeto_membros.loc[i,'Nome do Projeto de Pesquisa'] = df_projeto_membros.loc[i,'Nome do Projeto de Pesquisa'].replace('"','')

In [11]:
projetos_financiados = df_projeto_financiadores['Nome do Projeto de Pesquisa'].unique().tolist()
projetos_com_membros = df_projeto_membros['Nome do Projeto de Pesquisa'].unique().tolist()
projetos_todos = list(set(projetos_financiados + projetos_com_membros))

In [12]:
df = pd.DataFrame({
    'Projeto': projetos_todos
})

In [15]:
for i, row in df.iterrows():
    df.loc[i, 'Financiado'] = row.Projeto in df_projeto_financiadores['Nome do Projeto de Pesquisa'].tolist()

In [19]:
for i, row in df.iterrows():
    df_query = df_projeto_membros.loc[df_projeto_membros["Nome do Projeto de Pesquisa"] == row.Projeto]
    df.loc[i, 'Bolsista'] = 'Discente' in df_query['Categoria do Membro do Projeto'].tolist()